# A basic model to try and predict the Total Return of Eligible Stocks:
### Predicts the P/E multiple and hence the forecast price -> total return of a group of stocks

## Set-Up

### Import Packages

In [274]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.optimize import root
import matplotlib.pyplot as plt

### Import also the custom packages we made for this repository

In [275]:
__file__ = "NB04_Total_Return_Predictor.ipynb"
current_dir = os.path.dirname(os.path.abspath(__file__))
parent_dir = os.path.abspath(os.path.join(current_dir, '..', '../src'))
data_dir = os.path.abspath(os.path.join(current_dir, '..', '../data'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
from econ_utils import inv_theory as inv
from econ_utils import sql_queries as sqlq

### Set Starting Parameters

In [276]:
R = 0.055
b = 1/(1+R)
maturity_g = 0.073
time_frame = 12
start_earnings = 10



## Import the Bloomberg Rankings from Excel

### Read In The Data

In [277]:
asset_df = pd.read_excel(os.path.join(current_dir, "../../data/Bloomberg_Rankings.xlsx"), sheet_name = "Mid Cap and Above")

### Inspect the Data

In [278]:
asset_df.head(8)

,ID,Name,Last Price,6Y Total Return%,6Y Annualised Return,EPS,FCFPS,FCF Yield,EPS Growth,5 YR EPS Growth,5 YR Forecast EPS Growth,Multiple,Earnings Yield,EPS 5 Years Forecast,5Y Forecast Earnings Yield,1Y Expected TR (No Change in Multiple),5Y Expected Annualised TR (No Change in Multiple)
0,IHC UH Equity,International Holding Co PJSC,410.000000,300.470588,1.589450,13.526731,4.789175,1.168091e-02,88.549203,336.721739,336.721739,43.264225,0.023114,21489.034753,52.412280,3.468160,4.891563
1,QNBFB TI Equity,QNB Finansbank AS,328.750000,72.222393,1.045380,10.867053,NaN,NaN,69.538759,92.741867,92.741867,30.251914,0.033056,289.062723,0.879278,0.991131,1.212056
2,ADANIGR IN Equity,Adani Green Energy Ltd,2038.000000,63.698413,1.003621,6.944302,-74.834324,-3.671949e-02,12.776022,NaN,NaN,231.698702,0.004316,NaN,NaN,NaN,NaN
3,ADANI IN Equity,Adani Power Ltd,874.500000,46.398374,0.902365,54.003407,45.250976,5.174497e-02,94.187008,NaN,NaN,16.277971,0.061433,NaN,NaN,NaN,NaN
4,ADE IN Equity,Adani Enterprises Ltd,3645.250000,43.643866,0.883477,28.427895,-131.325263,-3.602641e-02,30.473393,52.594889,52.594889,137.441817,0.007276,235.203967,0.064523,0.537051,0.662944
5,SMCI US Equity,Super Micro Computer Inc,762.490723,30.004074,0.772433,19.230000,-33.824639,-4.436072e-02,72.776280,80.841010,80.841010,42.665360,0.023438,371.932307,0.487786,0.850796,1.045355
6,BYAN IJ Equity,Bayan Resources Tbk PT,18000.000000,22.218526,0.689037,0.034590,0.017968,9.982250e-07,-46.202778,63.130366,63.130366,32.120832,0.031132,0.399599,0.000022,0.682090,0.810109
7,6920 JP Equity,Lasertec Corp,40100.000000,21.450892,0.679599,742.560000,321.568675,8.019169e-03,105.626938,71.924868,71.924868,54.047060,0.018502,11153.856393,0.278151,0.751059,0.923087


### Drop the rows where "5Y Forecast EPS Growth" is Null
- We need this for our prediction of the P/E multiple, and then the EPS with which we multiply that multiple by to get the future price. 
- The current rudimentary form of predicting the multiple assumes that investors presume that EPS growth continues at the current rate for a certain set of years and then trails back down to a maturity growth rate. Investors purchase up until the point at which the multiple that they have priced in yields them the minimum expected rate of return, `R`, that they will take.
- I.e., the multiple today depends on the price and hence multiple tomorrow and so on so forth in a geometric series.
- Investors could calculate this `R` in any way, whether it be the CAPM or the Fama-French Three-Factor or another model to identify the best hurdle rate/ cost of equity capital they shoud take

In [279]:
asset_df = asset_df.dropna(subset=['5 YR Forecast EPS Growth'])
# Inspect the column (These figures will be in percentage, not proportion scale)
asset_df["5 YR Forecast EPS Growth"]


0       336.721739
1        92.741867
4        52.594889
5        80.841010
6        63.130366
           ...    
1033     12.294384
1039    -10.572361
1042      8.237525
1045     57.864844
1047     25.676972
Name: 5 YR Forecast EPS Growth, Length: 709, dtype: float64

In [280]:

def predict_multiple(growth, t, detail = False, max = 50): 
    growth = growth/100
    maturity_g = 0.065 + 0.1*(growth - 0.065)
    k_vec_m = 1 + maturity_g
    maturity_multiple = (k_vec_m*b*(1-(k_vec_m*b)**time_frame)/(1-b*k_vec_m))
    maturity_multiple   
    b_vec = [b] * t
    k_vec = [1 + growth] * t
    b_vec = np.cumprod(b_vec)[:t]
    k_vec = np.cumprod(k_vec)[:t]
    if detail:
        print(f'Maturity growth rate is assumed to be: {maturity_g}')
        print(f'As a result of the pdv of eps this results in, the maturity multiple is assumed to be: {maturity_multiple}')
        print(f'Pricing in an expected return of {R*100}%, we expect an end of period p/e of...')   
    return float(np.min([sum(k_vec*b_vec) + maturity_multiple*b_vec[-1]*k_vec[-1], max]))


predict_multiple(14,5, detail = True)

Maturity growth rate is assumed to be: 0.07250000000000001
As a result of the pdv of eps this results in, the maturity multiple is assumed to be: 13.375895034674066
Pricing in an expected return of 5.5%, we expect an end of period p/e of...


26.051838189309326

In [281]:
asset_df["5Y Multiple"] = asset_df["5 YR Forecast EPS Growth"].apply(lambda x: predict_multiple(x, 5))


In [282]:
def predict_roi(growth, multiple, fut_multiple, n = 3, payout_rate = 1, detail = False): 
    growth = min([growth/100,0.30])
    fut_price_to_curr_price = ((1+growth)**n)*(fut_multiple/multiple)
    growth_vec = [1+ growth] * n
    cum_growth_vec = np.cumprod(growth_vec)
    cum_income_to_curr_price = (1/multiple)*(np.sum(cum_growth_vec))
    pr = fut_price_to_curr_price
    ir = float(cum_income_to_curr_price)
    tr = pr + ir
    tr_ann = float(tr**(1/n) - 1)
    if detail:
        print(f'Estimated geometric average annual eps growth in the period: {growth*100}%')
        print(f'Cumulative eps growth in the period: {(cum_growth_vec[-1]-1)*100}%')
        print(f'And the P/E is expected to change to: {fut_multiple} from its current {multiple}')
        print(f'Price reaches {(pr)*100}% of the starting price')
        print(f'Cumulative income is {ir*100}% of the starting price')
        print(f'Total Return is {tr*100}%')
        print(f'Annualised to a {n} year period is {tr_ann*100}%')
    return tr_ann 
predict_roi(10, 6, predict_multiple(10,5), detail = True)

Estimated geometric average annual eps growth in the period: 10.0%
Cumulative eps growth in the period: 33.10000000000004%
And the P/E is expected to change to: 21.753977144561347 from its current 6
Price reaches 482.5757263235193% of the starting price
Cumulative income is 60.683333333333344% of the starting price
Total Return is 543.2590596568526%
Annualised to a 3 year period is 75.79330285037489%


0.7579330285037489

## Predict the ROI Corresponding to this

In [283]:

asset_df["5Y Expected Annualised TR (Change in Multiple)"] = [predict_roi(asset_df["5 YR Forecast EPS Growth"][i], asset_df["Multiple"][i], asset_df["5Y Multiple"][i]) for i in asset_df.index]

In [284]:
asset_df.head()

,ID,Name,Last Price,6Y Total Return%,6Y Annualised Return,EPS,FCFPS,FCF Yield,EPS Growth,5 YR EPS Growth,5 YR Forecast EPS Growth,Multiple,Earnings Yield,EPS 5 Years Forecast,5Y Forecast Earnings Yield,1Y Expected TR (No Change in Multiple),5Y Expected Annualised TR (No Change in Multiple),5Y Multiple,5Y Expected Annualised TR (Change in Multiple)
0,IHC UH Equity,International Holding Co PJSC,410.000000,300.470588,1.589450,13.526731,4.789175,1.168091e-02,88.549203,336.721739,336.721739,43.264225,0.023114,21489.034753,52.412280,3.468160,4.891563,50.0,0.385382
1,QNBFB TI Equity,QNB Finansbank AS,328.750000,72.222393,1.045380,10.867053,NaN,NaN,69.538759,92.741867,92.741867,30.251914,0.033056,289.062723,0.879278,0.991131,1.212056,50.0,0.560851
4,ADE IN Equity,Adani Enterprises Ltd,3645.250000,43.643866,0.883477,28.427895,-131.325263,-3.602641e-02,30.473393,52.594889,52.594889,137.441817,0.007276,235.203967,0.064523,0.537051,0.662944,50.0,-0.057591
5,SMCI US Equity,Super Micro Computer Inc,762.490723,30.004074,0.772433,19.230000,-33.824639,-4.436072e-02,72.776280,80.841010,80.841010,42.665360,0.023438,371.932307,0.487786,0.850796,1.045355,50.0,0.391834
6,BYAN IJ Equity,Bayan Resources Tbk PT,18000.000000,22.218526,0.689037,0.034590,0.017968,9.982250e-07,-46.202778,63.130366,63.130366,32.120832,0.031132,0.399599,0.000022,0.682090,0.810109,50.0,0.529972


In [285]:
asset_df.to_excel(os.path.join(current_dir,"../../data/Top_Stocks.xlsx"))

## Re-do with more complicated bloomberg data

### Import the data

#### Snapshot Data

In [286]:
df = pd.read_sql("SELECT * FROM 'table_2'", sqlq.engine)

#### Historic Data

In [287]:
#df_2 = pd.read_excel(os.path.join(current_dir, "../../data/bloomberg_data.xlsx"))


### Rename Some of the columns

In [288]:
df.rename(columns = {'Unnamed: 0': 'security_key'}, inplace = True)

### Take a deeper look at some of the columns, especially annualised performance over the last 5 years

In [289]:
pd.set_option('display.max_columns', None)
df.sort_values(by = "current_ann_trr_5yr", ascending = False)

,security_key,security_name,px_last,current_trr_ytd,cur_mkt_cap,current_trr_ytd_1,current_ann_trr_3yr,current_ann_trr_5yr,month_end_trr_10yr,minimum_total_return,hist_trr_prev_1yr,current_trr_mtd,eps_growth,geo_grow_diluted_eps_cont_ops,diluted_eps_cont_ops_5yr_avg_gr,5y_geo_growth_diluted_eps,earn_yld,pe_ratio,five_yr_avg_price_earnings,long_term_price_earnings_ratio,best_pe_ratio,pr_eps_growth_plus_yld
182,IHC UH Equity,International Holding Co PJSC,414.500,3.754693,9.092223e+11,3.754693,48.553680,209.09870,79.483020,NaN,-2.560967,1.593137,84.368610,196.464049,280.792101,215.717287,3.263385,30.643028,34.160428,NaN,NaN,NaN
240,NVDA US Equity,NVIDIA Corp,125.830,154.121900,3.095418e+12,154.121900,84.915230,100.09750,75.716730,NaN,239.019100,1.853651,584.659091,51.441971,168.466403,48.401189,1.364966,73.261887,61.097322,366.074157,43.136784,42.817484
309,TSLA US Equity,Tesla Inc,251.520,1.223442,8.021466e+11,1.223442,4.942127,74.82077,28.611223,NaN,101.721000,27.107340,17.661692,NaN,NaN,NaN,0.883704,113.160080,NaN,405.377228,94.449869,NaN
203,LLY US Equity,Eli Lilly & Co,914.570,57.423540,8.692123e+11,57.423540,58.498100,54.49960,33.149180,NaN,60.905030,1.015044,-16.017316,13.459286,22.494442,13.129725,1.133808,88.198379,38.449955,135.566601,62.526150,40.544444
199,KLAC US Equity,KLA Corp,855.210,47.740160,1.151454e+11,47.740160,42.903900,50.77105,33.369447,NaN,56.029640,3.723425,10.013593,25.827603,25.775781,36.480119,2.637979,37.907804,17.870855,62.973129,31.722616,9.645935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,P911 GR Equity,Dr Ing hc F Porsche AG,71.320,-7.935353,6.497252e+10,-7.935353,NaN,NaN,NaN,NaN,-14.915430,2.589187,4.044118,NaN,NaN,NaN,7.182267,13.923180,NaN,NaN,12.298672,6.568923
255,PLTR US Equity,Palantir Technologies Inc,27.230,58.590560,6.063839e+10,58.590560,5.661175,NaN,NaN,NaN,167.445400,7.500986,NaN,NaN,NaN,NaN,0.461469,216.699291,NaN,NaN,79.619883,NaN
258,PRX NA Equity,Prosus NV,32.835,21.678700,8.467185e+10,21.678700,-2.140393,NaN,NaN,NaN,-8.503771,-1.277809,62.145326,10.844301,75.982422,20.394921,7.447539,13.427254,18.448145,NaN,11.597772,46.123187
272,ROSN RM Equity,Rosneft Oil Co PJSC,556.150,NaN,5.894177e+12,NaN,NaN,NaN,14.568560,NaN,79.459110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.225822,NaN,NaN


### Fill in blanks for 5 year geometric average earnings growth to get return calc

In [290]:
def clean_and_fill(df):
    """
    Remove rows with NaN values across all specified columns, and fill NaNs in the 
    '5y_geo_growth_diluted_eps' column with the first available non-null value from other columns.

    Parameters:
    - df (DataFrame): The DataFrame to process.

    Returns:
    - DataFrame: The processed DataFrame.
    """
    # Define the columns to check for NaN values
    columns_to_check = [
        'eps_growth', 
        'geo_grow_diluted_eps_cont_ops', 
        'diluted_eps_cont_ops_5yr_avg_gr', 
        '5y_geo_growth_diluted_eps'
    ]
    
    # Drop rows where all specified columns are NaN and create a copy
    df_cleaned = df.dropna(subset=columns_to_check, how='all').copy()
    
    # Fill NaNs in '5y_geo_growth_diluted_eps' with the first available non-null value
    df_cleaned['5y_geo_growth_diluted_eps'] = df_cleaned['5y_geo_growth_diluted_eps'].fillna(
        df_cleaned['diluted_eps_cont_ops_5yr_avg_gr']
    ).fillna(
        df_cleaned['geo_grow_diluted_eps_cont_ops']
    ).fillna(
        df_cleaned['eps_growth']
    )
    
    # Merge the cleaned '5y_geo_growth_diluted_eps' back into the original df
    df_merged = df.merge(
        df_cleaned[['5y_geo_growth_diluted_eps']], 
        left_index=True, 
        right_index=True, 
        how='left', 
        suffixes=('', '_filled')
    )
    
    # Use the filled values to update the original column
    df_merged['5y_geo_growth_diluted_eps'] = df_merged['5y_geo_growth_diluted_eps_filled']

    # Drop the extra column after merging
    df_merged.drop(columns=['5y_geo_growth_diluted_eps_filled'], inplace=True)

    return df_merged

df = clean_and_fill(df)

In [291]:
pd.set_option('display.max_rows', 10)
df

,security_key,security_name,px_last,current_trr_ytd,cur_mkt_cap,current_trr_ytd_1,current_ann_trr_3yr,current_ann_trr_5yr,month_end_trr_10yr,minimum_total_return,hist_trr_prev_1yr,current_trr_mtd,eps_growth,geo_grow_diluted_eps_cont_ops,diluted_eps_cont_ops_5yr_avg_gr,5y_geo_growth_diluted_eps,earn_yld,pe_ratio,five_yr_avg_price_earnings,long_term_price_earnings_ratio,best_pe_ratio,pr_eps_growth_plus_yld
0,000333 CH Equity,Midea Group Co Ltd,64.00,22.189810,4.467274e+11,22.189810,1.145894,6.886530,21.267630,NaN,10.856210,-0.775194,11.248249,9.349578,9.054256,9.621721,7.780530,12.852595,16.435892,18.269357,NaN,NaN
1,000660 KS Equity,SK Hynix Inc,236000.00,67.065220,1.718086e+14,67.065220,26.322820,29.949640,18.934220,NaN,90.694470,-0.211417,NaN,NaN,NaN,NaN,-2.836933,NaN,NaN,29.344968,7.878897,42.571356
2,000858 CH Equity,Wuliangye Yibin Co Ltd,128.95,-8.096358,5.005334e+11,-8.096358,-20.547550,1.532926,24.229110,NaN,-20.579780,0.710718,13.190809,17.636014,5.300772,17.506920,6.335673,15.783643,33.542675,28.014891,NaN,13.642920
3,005930 KS Equity,Samsung Electronics Co Ltd,87100.00,11.451140,5.199681e+14,11.451140,4.956467,17.323510,14.683610,NaN,44.898240,6.871166,-73.550949,-19.810062,78.530824,-19.895532,3.329506,30.034482,19.193506,17.522041,13.515566,63.123398
4,1088 HK Equity,China Shenhua Energy Co Ltd,37.35,49.269520,9.256450e+11,49.269520,50.082570,31.524360,15.390870,NaN,32.600360,3.894291,-11.362398,7.717313,10.502693,7.950673,8.938004,11.188180,6.399881,14.706056,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,WMT US Equity,Walmart Inc,70.04,34.194950,5.633698e+11,34.194950,16.329720,15.058430,12.721773,NaN,12.885850,3.441149,34.265734,6.222695,12.018819,20.450675,3.332766,30.005111,23.495331,33.485832,28.310428,9.365228
340,XLK US Equity,Technology Select Sector SPDR,232.88,21.406060,7.253185e+10,21.406060,16.810780,25.201440,20.971694,-6.27495,56.016360,2.939491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341,XOM US Equity,Exxon Mobil Corp,113.37,15.379720,5.085696e+11,15.379720,28.476220,13.750990,5.789788,NaN,-6.243449,-1.520153,-32.956259,14.081479,NaN,12.744766,7.714310,12.962921,NaN,18.606106,11.945001,9.316867
342,ZTS US Equity,Zoetis Inc,174.96,-10.902600,7.983340e+10,-10.902600,-3.100194,9.685760,19.094823,NaN,35.899130,0.922938,12.638581,11.126667,10.580528,11.590717,3.021147,33.100011,39.442777,44.330586,29.836289,11.282233


### Use the multiple predictor forecast total returns and later rank by forecast total returns


In [292]:
def gen_returns(df, decr = 0.7, factor = 0.5):
    df["exp_geo_growth_diluted_eps"] = decr*df["5y_geo_growth_diluted_eps"]
    df["exp_pe_ratio"] = df["exp_geo_growth_diluted_eps"].apply(lambda x: predict_multiple(x, 5))
    df["exp_pe_ratio_lb"] = [max([elem, 8]) for elem in factor*df["pe_ratio"]]
    df["exp_ann_trr"] = [predict_roi(df["exp_geo_growth_diluted_eps"][i], df["pe_ratio"][i], df["exp_pe_ratio"][i]) for i in df.index]
    df["exp_ann_trr"] = df["exp_ann_trr"].fillna(decr*df["current_ann_trr_5yr"]/100)
    df["exp_ann_trr_lb"] = [predict_roi(df["exp_geo_growth_diluted_eps"][i], df["pe_ratio"][i], df["exp_pe_ratio_lb"][i]) for i in df.index]
    df["exp_ann_trr_lb"] = df["exp_ann_trr_lb"].fillna(decr*factor*df["current_ann_trr_5yr"]/100)
    return df
df = gen_returns(df)
df.to_excel(os.path.join(data_dir, "unfiltered_basket.xlsx"))

### Define a function to apply some exclusion criteria
* Get rid of any stocks that were in the bottom 30% of performers in the 1yr, 3yr and 5yr annualised TR
* Get rid of any stocks with EPS growth in the last year, geometrically over 5 years or average over 5 years in the bottom 20 % of performers

In [293]:
def exclude(df, tr_floor=0, eps_g_floor=0.0, tr_pctl_floor=5, eps_g_pctl_floor=5, five_yr_ann_eps_g_min=0, exp_trr_lb_floor = 0.06):
    """
    Filter DataFrame rows based on conditions applied to columns containing 'trr' or growth-related terms.

    Parameters:
    - df (DataFrame): The DataFrame to filter.
    - tr_floor (float): Minimum threshold for 'trr' columns.
    - eps_g_floor (float): Minimum threshold for EPS growth columns.
    - tr_pctl_floor (float): Percentile threshold for 'trr' columns.
    - eps_g_pctl_floor (float): Percentile threshold for EPS growth columns.
    - five_yr_ann_eps_g_min (float): Not used in this version, reserved for future conditions.

    Returns:
    - DataFrame: The filtered DataFrame.
    """
    
    # Filter columns matching the regex for 'trr'
    trr_cols = df.filter(regex='trr')
    for col in trr_cols.columns:
        # Compute the given percentile for each 'trr' column
        percentile = np.nanpercentile(df[col], tr_pctl_floor)
        
        # Create a mask to filter rows where each 'trr' column is either NaN or meets conditions
        mask = (df[col].isna()) | ((df[col] >= percentile) & (df[col] >= tr_floor))
        
        # Apply the mask to filter the DataFrame for each 'trr' column
        df = df[mask]

    # Filter columns matching the regex for 'eps' or 'earnings' combined with 'gr'
    eps_growth_cols = df.filter(regex='(gr.*eps|eps.*gr|gr.*earnings|earnings.*gr)')
    for col in eps_growth_cols.columns:
        # Compute the given percentile for each growth-related column
        percentile = np.nanpercentile(df[col], eps_g_pctl_floor)

        # Create a mask to filter rows where each column is either NaN or meets conditions
        mask = (df[col].isna()) | ((df[col] >= percentile) & (df[col] >= eps_g_floor))

        # Apply the mask to filter the DataFrame for each column
        df = df[mask]

    # Eliminate stocks that have a chance of performing below a threshold
    mask = (df["exp_ann_trr_lb"].isna()) | (df["exp_ann_trr_lb"] <= exp_trr_lb_floor)
     # Rank by total returns and select up to top 50
    df = df.sort_values(by = "exp_ann_trr", ascending = False)
    
    
    return df.head(50)

# Example usage
filtered_df = exclude(df)
filtered_df



,security_key,security_name,px_last,current_trr_ytd,cur_mkt_cap,current_trr_ytd_1,current_ann_trr_3yr,current_ann_trr_5yr,month_end_trr_10yr,minimum_total_return,hist_trr_prev_1yr,current_trr_mtd,eps_growth,geo_grow_diluted_eps_cont_ops,diluted_eps_cont_ops_5yr_avg_gr,5y_geo_growth_diluted_eps,earn_yld,pe_ratio,five_yr_avg_price_earnings,long_term_price_earnings_ratio,best_pe_ratio,pr_eps_growth_plus_yld,exp_geo_growth_diluted_eps,exp_pe_ratio,exp_pe_ratio_lb,exp_ann_trr,exp_ann_trr_lb
281,SBIN IN Equity,State Bank of India,859.75,36.14766,7.672935e+12,36.14766,28.683860,20.758480,13.45286,NaN,6.658328,1.272158,20.561347,95.958450,121.817039,96.284031,8.743239,11.437409,10.850982,29.051121,NaN,NaN,67.398822,50.000000,8.000000,1.158589,0.257865
129,CS FP Equity,AXA SA,31.96,14.99040,7.259724e+10,14.99040,21.742890,12.263980,11.53627,NaN,20.141520,4.546939,47.417840,8.239508,320.191972,31.699295,9.843004,10.159500,12.056767,13.387884,9.255720,14.595244,22.189506,37.279381,8.000000,0.925689,0.234932
280,SBER RM Equity,Sberbank of Russia PJSC,313.50,NaN,6.767508e+12,NaN,NaN,NaN,19.08770,NaN,114.086700,NaN,246.924885,12.248339,NaN,12.596420,22.606001,4.423604,6.049010,6.381916,NaN,NaN,8.817494,20.611128,8.000000,0.895344,0.463606
179,ICICIBC IN Equity,ICICI Bank Ltd,1233.40,23.76079,8.678435e+12,23.76079,25.004470,24.150440,17.69244,NaN,12.790760,2.817610,29.328694,62.597397,106.664939,56.833793,5.123236,19.518912,19.727694,35.477266,NaN,NaN,39.783655,50.000000,9.759456,0.806317,0.109082
60,ABNB US Equity,Airbnb Inc,152.50,12.01704,9.821668e+10,12.01704,2.277222,NaN,NaN,NaN,59.228070,0.573762,153.198653,NaN,NaN,153.198653,4.891241,20.444708,NaN,NaN,31.314168,NaN,107.239057,50.000000,10.222354,0.778630,0.105812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,VEA US Equity,Vanguard FTSE Developed Market,50.48,6.99156,1.356206e+11,6.99156,2.931059,7.041403,4.54140,-3.2307,17.937110,2.144883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049290,0.024645
105,BREN IJ Equity,Barito Renewables Energy Tbk P,10300.00,37.83897,1.377998e+15,37.83897,NaN,NaN,NaN,NaN,NaN,2.233251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,CEG US Equity,Constellation Energy Corp,211.29,81.40372,6.658190e+10,81.40372,NaN,NaN,NaN,NaN,37.233570,5.502566,NaN,NaN,NaN,NaN,3.288681,30.407327,NaN,NaN,27.440260,15.190596,NaN,NaN,15.203663,NaN,NaN
255,PLTR US Equity,Palantir Technologies Inc,27.23,58.59056,6.063839e+10,58.59056,5.661175,NaN,NaN,NaN,167.445400,7.500986,NaN,NaN,NaN,NaN,0.461469,216.699291,NaN,NaN,79.619883,NaN,NaN,NaN,108.349646,NaN,NaN
